In [1]:
import pandas as pd
import glob
import os
import sys
sys.path.append(os.path.abspath(os.path.join('..')))
from config import *

In [2]:
#path to txt files
file_path_cases = output_path+'/cases/*multianno.txt'
#list all txt files
cases = glob.glob(file_path_cases)

file_path_controls = output_path+'/control/*multianno.txt'
control = glob.glob(file_path_controls)

In [ ]:
categories = ['PTPN22','LACC1','NOD2','TYK2','IL23R']  # Only consider the exonic category
final_results = pd.DataFrame(columns=['patient_id'] + categories + ['label'])

for file in cases + control:
    df = pd.read_csv(file, sep="\t", low_memory=False)
    patient_id = file.split("/")[-1].replace(".hg19_multianno.txt", "")
    function_col = 'Func.refGene'
    genes_col = 'Gene.refGene'
    gt = df.columns[-1]
    df['variant_count'] = df[gt].apply(lambda x: 2 if x.startswith('1/1') else 1)
    variant_counts = df.groupby(function_col)['variant_count'].sum().reindex(categories, fill_value=0)
    label = 1 if "cases" in file else 0
    result_row = pd.DataFrame([[patient_id] + list(variant_counts) + [label]], columns=['patient_id'] + categories + ['label'])
    final_results = pd.concat([final_results, result_row], ignore_index=True)
#final_results = final_results.sample(frac=1, random_state=42).reset_index(drop=True)
final_results.to_csv(vcf_path + "/ExonicVariantsCounts.csv", index=False)
print(final_results)

    patient_id exonic label
0          CD1      4     1
1         CD10      5     1
2         CD11      3     1
3         CD12      4     1
4         CD13      3     1
..         ...    ...   ...
106        HC5      2     0
107        HC6      3     0
108        HC7      2     0
109        HC8      3     0
110        HC9      2     0

[111 rows x 3 columns]


In [4]:
categories = ['PTPN22', 'LACC1', 'NOD2', 'TYK2', 'IL23R']  # Genes to consider
final_results = pd.DataFrame(columns=['patient_id'] + categories + ['label'])

for file in cases + control:
    df = pd.read_csv(file, sep="\t", low_memory=False)
    patient_id = file.split("/")[-1].replace(".hg19_multianno.txt", "")
    function_col = 'Func.refGene'
    genes_col = 'Gene.refGene'
    gt = df.columns[-1]

    # Apply lambda to assign variant counts: 2 for homozygous (1/1), 1 for heterozygous (1/0 or 0/1)
    df['variant_count'] = df[gt].apply(lambda x: 2 if x.startswith('1/1') else 1)

    # Group by gene and calculate the sum of variant counts
    gene_variant_counts = df.groupby(genes_col)['variant_count'].sum()

    # Initialize variant counts for all genes, defaulting to 0
    variant_counts = {gene: 0 for gene in categories}

    # Update the variant counts for the genes present in the current file
    for gene in categories:
        if gene in gene_variant_counts.index:
            variant_counts[gene] = gene_variant_counts[gene]

    # Set label: 1 for cases, 0 for control
    label = 1 if "cases" in file else 0

    # Create a row for the current patient
    result_row = pd.DataFrame([[patient_id] + list(variant_counts.values()) + [label]], columns=['patient_id'] + categories + ['label'])

    # Append the result to the final dataframe
    final_results = pd.concat([final_results, result_row], ignore_index=True)

# Shuffle the results and reset the index
final_results = final_results.sample(frac=1, random_state=42).reset_index(drop=True)

# Save the final results to CSV
final_results.to_csv(vcf_path + "/GeneVariantsCounts.csv", index=False)

# Print the final results
print(final_results)


    patient_id PTPN22 LACC1 NOD2 TYK2 IL23R label
0         HC22      2     1    1    0     0     0
1         CD19      2     1    0    1     0     1
2         CD13      2     1    0    0     0     1
3         HC28      2     2    0    0     1     0
4          HC1      2     0    0    0     0     0
..         ...    ...   ...  ...  ...   ...   ...
106        HC5      2     0    0    0     0     0
107       CD22      2     0    0    0     1     1
108       HC35      1     1    0    0     0     0
109       CD56      2     2    0    0     0     1
110       HC44      2     1    0    0     0     0

[111 rows x 7 columns]
